<a href="https://colab.research.google.com/github/Arya-Hari/fabric-classification-cnn/blob/main/model2a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp drive/MyDrive/my_utils.py .
import my_utils

In [ ]:
#based of https://www.tensorflow.org/tutorials/load_data/images
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import re

import pathlib

path = "/content/drive/MyDrive/Fabrics_simple"
data_dir = pathlib.Path(path)

In [ ]:
batch_size = 32
img_height = 400
img_width = 400

train_ds, val_ds, class_names = my_utils.generate_ds(data_dir, batch_size, img_height, img_width)

print(class_names)

Found 1008 files belonging to 2 classes.
Using 807 files for training.
Found 1008 files belonging to 2 classes.
Using 201 files for validation.
['Denim', 'Wool']


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
#based of https://www.tensorflow.org/tutorials/load_data/images
num_classes = 2

model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Conv2D(32, 3, input_shape=(32, 400, 400, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

checkpoint = my_utils.generate_callback("/content/drive/MyDrive/fabric_classification/", "Model-2a-NoAug-fs")

hist = model.fit(
    train_ds,
    validation_data = val_ds,
    epochs = 50,
    verbose = 1,
    callbacks = [checkpoint])

model.summary()

Epoch 1/50
26/26 [==============================] - ETA: 0s - loss: 1.1633 - accuracy: 0.6320
Epoch 1: val_accuracy improved from -inf to 0.73134, saving model to /content/drive/MyDrive/fabric_classification/Model-2a-NoAug-fs.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


26/26 [==============================] - 340s 10s/step - loss: 1.1633 - accuracy: 0.6320 - val_loss: 0.5304 - val_accuracy: 0.7313
Epoch 2/50
26/26 [==============================] - ETA: 0s - loss: 0.5706 - accuracy: 0.7299
Epoch 2: val_accuracy improved from 0.73134 to 0.79602, saving model to /content/drive/MyDrive/fabric_classification/Model-2a-NoAug-fs.h5
26/26 [==============================] - 193s 7s/step - loss: 0.5706 - accuracy: 0.7299 - val_loss: 0.4600 - val_accuracy: 0.7960
Epoch 3/50
26/26 [==============================] - ETA: 0s - loss: 0.4905 - accuracy: 0.7695
Epoch 3: val_accuracy improved from 0.79602 to 0.87065, saving model to /content/drive/MyDrive/fabric_classification/Model-2a-NoAug-fs.h5
26/26 [==============================] - 193s 7s/step - loss: 0.4905 - accuracy: 0.7695 - val_loss: 0.4268 - val_accuracy: 0.8706
Epoch 4/50
26/26 [==============================] - ETA: 0s - loss: 0.3125 - accuracy: 0.8786
Epoch 4: val_accuracy improved from 0.87065 to 0.89

In [ ]:
my_utils.generate_plot(hist)

In [ ]:

sunflower_path = "/content/drive/MyDrive/im_1.png"

img = tf.keras.utils.load_img(
    sunflower_path, target_size=(img_height, img_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

In [ ]:

sunflower_path = "/content/drive/MyDrive/blue-denim-fabric.jpg"

img = tf.keras.utils.load_img(
    sunflower_path, target_size=(img_height, img_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)